In [1]:
import yfinance
import numpy as np
import pandas as pd
import pandas
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import cufflinks as cf
cf.set_config_file(offline=True)

import warnings
warnings.filterwarnings("ignore")


In [2]:
from ActMng import AccountManager

In [3]:
starting_capital = 9000
allocation = 1

symbol = 'DOT1-USD'

long_trend_days = 12
short_trend_days = 6

In [4]:
def fetch_data(symbol:str='DOT1-USD', start_date:str='2020-8-20', end_date:str=None):
    
    if end_date is None:
        end = pandas.Timestamp.now()
    else:
        end = end_date
    
    return pdr.get_data_yahoo(symbol, pandas.Timestamp(start_date), pandas.Timestamp(end))[['Open','Close']]
df = fetch_data(symbol)

[*********************100%***********************]  1 of 1 completed


In [5]:
fast_trend = df.Open.rolling(window=short_trend_days).mean()
slow_trend = df.Open.rolling(window=long_trend_days).mean()

df['ma_fast'] = fast_trend
df['ma_slow'] = slow_trend
df[['Close','ma_fast','ma_slow']].iplot()

In [6]:
df['trade_direction'] = np.sign(fast_trend-slow_trend)
df.head(20)

,Open,Close,ma_fast,ma_slow,trade_direction
Date,,,,,
2020-08-20,2.787857,2.900080,NaN,NaN,NaN
2020-08-21,2.896923,2.875028,NaN,NaN,NaN
2020-08-22,2.878570,4.484690,NaN,NaN,NaN
2020-08-23,4.487058,3.967066,NaN,NaN,NaN
2020-08-24,3.987413,4.602614,NaN,NaN,NaN
2020-08-25,4.584953,5.523393,3.603796,NaN,NaN
2020-08-26,5.541406,6.081716,4.062721,NaN,NaN
2020-08-27,6.075772,5.628337,4.592529,NaN,NaN
2020-08-28,5.639486,6.159955,5.052681,NaN,NaN


In [7]:
#Determine the direction of the moving average differences
df_trades = df['trade_direction'].dropna().diff().dropna().apply(np.sign).astype('int8')
df_trades = pandas.DataFrame(df_trades[df_trades !=0])
df_trades
#Label our intended trades
df_trades['trade'] = df['trade_direction'].map({1:'Long',-1:'Short'})
df['trades'] = df_trades['trade']
df_trades
#df['trades'].head(30)
#type(df['trades'])
#df_trades['trade']
#df['trade_direction']

,trade_direction,trade
Date,,
2020-09-23,-1,Short
2020-10-06,1,Long
2020-10-07,-1,Short
2020-10-17,1,Long
2020-10-20,-1,Short
2020-10-25,1,Long
2020-11-02,-1,Short
2020-11-09,1,Long
2020-11-30,-1,Short


In [8]:
#Label our opening trade after removing our window period
df['trades'].replace(pandas.np.NaN,'None',inplace=True)
df = df.dropna()
df['trades'].iloc[0]=('Long' if df['trade_direction'].iloc[0]==1.0 else 'Short')
df

,Open,Close,ma_fast,ma_slow,trade_direction,trades
Date,,,,,,
2020-08-31,5.905918,6.300020,5.915324,4.759560,1.0,Long
2020-09-01,6.298810,6.288767,6.041558,5.052139,1.0,None
2020-09-02,6.282686,6.139465,6.076044,5.334286,1.0,None
2020-09-03,6.131179,5.747868,6.157993,5.605337,1.0,None
2020-09-04,5.726610,5.146038,6.083107,5.708633,1.0,None
...,...,...,...,...,...,...
2021-11-18,42.734184,39.045967,44.648632,47.567895,-1.0,None
2021-11-19,39.004200,41.261021,43.498397,46.478601,-1.0,None
2021-11-20,41.257370,42.060429,42.518766,45.560115,-1.0,None


In [9]:
# Our trading strategy implementation - feel free to brose ActMng.py if you are curious.
trading_strategy = AccountManager(symbol=symbol,trades_df=df,starting_capital=starting_capital,allocation_amount=allocation)
results = trading_strategy.run()

In [10]:
#Here we can see a day by day breakdown of the account
results['result_df']

,Open,Close,ma_fast,ma_slow,trade_direction,trades,account_val,cash_bal,position,position_changes,fees
2020-08-31,5.905918,6.300020,5.915324,4.759560,1.0,Long,9599.572185,-1.028625,1523.90,1523.90,1.0
2020-09-01,6.298810,6.288767,6.041558,5.052139,1.0,None,9582.423194,-1.028625,1523.90,0.00,0.0
2020-09-02,6.282686,6.139465,6.076044,5.334286,1.0,None,9354.901868,-1.028625,1523.90,0.00,0.0
2020-09-03,6.131179,5.747868,6.157993,5.605337,1.0,None,8758.147513,-1.028625,1523.90,0.00,0.0
2020-09-04,5.726610,5.146038,6.083107,5.708633,1.0,None,7841.018768,-1.028625,1523.90,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,42.734184,39.045967,44.648632,47.567895,-1.0,None,168.048112,285.576473,-3.01,0.00,0.0
2021-11-19,39.004200,41.261021,43.498397,46.478601,-1.0,None,161.380800,285.576473,-3.01,0.00,0.0
2021-11-20,41.257370,42.060429,42.518766,45.560115,-1.0,None,158.974582,285.576473,-3.01,0.00,0.0
2021-11-21,42.114777,42.005707,41.804416,44.621697,-1.0,None,159.139295,285.576473,-3.01,0.00,0.0


In [11]:
# Here are our simulated orders as we would submit to an API endpoint for algorthmic trading. 
#o=lenresults['broker_orders']['order'].loc[:]='BUY' 
 #.count()

results['broker_orders']



,symbol,date,time,order,order_type,price,size,exchange_fees
2020-08-31,DOT1-USD,2020-08-31 00:00:00,At Market Open,BUY,MARKET,5.905918,1523.90,1.0
2020-09-23,DOT1-USD,2020-09-23 00:00:00,At Market Open,SELL,MARKET,4.172700,1523.90,1.0
2020-09-23,DOT1-USD,2020-09-23 00:00:00,At Market Open,SELL,MARKET,4.172700,1523.41,1.0
2020-10-06,DOT1-USD,2020-10-06 00:00:00,At Market Open,BUY,MARKET,4.171682,1523.41,1.0
2020-10-06,DOT1-USD,2020-10-06 00:00:00,At Market Open,BUY,MARKET,4.171682,1523.68,1.0
...,...,...,...,...,...,...,...,...
2021-11-03,DOT1-USD,2021-11-03 00:00:00,At Market Open,BUY,MARKET,51.631992,4.72,1.0
2021-11-03,DOT1-USD,2021-11-03 00:00:00,At Market Open,BUY,MARKET,51.631992,3.06,1.0
2021-11-12,DOT1-USD,2021-11-12 00:00:00,At Market Open,SELL,MARKET,47.569553,3.06,1.0
2021-11-12,DOT1-USD,2021-11-12 00:00:00,At Market Open,SELL,MARKET,47.569553,3.01,1.0


In [12]:
r_df = results['result_df']

# Calculate the Absolute Return of our trend-following trading strategy
absolute_return_tf = (r_df['account_val'].iloc[-1]-starting_capital)/starting_capital

# Calculate the CAGR or our trend-following trading strategy
CAGR_tf = ((r_df['account_val'].iloc[-1]/starting_capital)**(252/len(df)))-1
#or
CAGR_tf = ((r_df['account_val'].iloc[-1]/starting_capital)**(365/(df.index[-1] - df.index[0]).days))-1


# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio_tf = (r_df.account_val.pct_change().mean()/r_df.account_val.pct_change().std())*((252) **0.5)

#Plot the equity curve (hint: use 'account_val')
r_df.account_val.iplot()
#or
#r_df.account_val.plot()


In [13]:
#Todo:
# Calculate the Absolute Return of a buy-and-hold strategy in the same period
absolute_return_bh = (r_df.Close.iloc[-1]-r_df.Open.iloc[0])/r_df.Open.iloc[0]
# Calculate the CAGR or a buy-and-hold trading strategy
CAGR_bh = ((r_df.Close.iloc[-1]/r_df.Open.iloc[0])**(252/len(df)))-1
#or
CAGR_bh = ((r_df.Close.iloc[-1]/r_df.Open.iloc[0])**(365/(df.index[-1] - df.index[0]).days))-1

# Calculate the IR (Sharpe without RF-rate) of a buy-and-hold trading strategy
information_ratio_bh = (r_df.Close.pct_change().mean()/r_df.Close.pct_change().std())*((252) **0.5)

#Plot the equity curve (hint: use 'close' returns and your initial capital value to generate an equity curve)
((r_df.Close.pct_change().dropna()+1).cumprod()*starting_capital).iplot()
#or
#((r_df.Close.pct_change().dropna()+1).cumprod()*starting_capital).plot()


In [14]:
#Todo - optional, below:
# Copy any of the code from above to new cells below, and see how changing the symbol, dates and trend parameters changes the performance
# Can you beat the reference implementation?

In [15]:
def fun_with_assignment_2(symbol=symbol,
                          allocation=0.8,
                          starting_capital=100000,
                          short_trend=6,
                          long_trend=12,
                          start_date='2020',
                          end_date=None):

#     starting_capital = 100000
#     allocation = 0.8

#     symbol = 'SPY'

#     long_trend_days = 12
#     short_trend_days = 6
    
    starting_capital = starting_capital
    allocation = allocation

    symbol = symbol

    long_trend_days = long_trend
    short_trend_days = short_trend
    
    df = fetch_data(symbol,start_date=start_date,end_date=end_date)
    
    fast_trend = df.Open.rolling(window=short_trend_days).mean()
    slow_trend = df.Open.rolling(window=long_trend_days).mean()

    df['ma_fast'] = fast_trend
    df['ma_slow'] = slow_trend
    df[['Close','ma_fast','ma_slow']].iplot()
    
    df['trade_direction'] = np.sign(fast_trend-slow_trend)
    
    #Determine the direction of the moving average differences
    df_trades = df['trade_direction'].dropna().diff().dropna().apply(np.sign).astype('int8')
    df_trades = pandas.DataFrame(df_trades[df_trades !=0])

    #Label our intended trades
    df_trades['trade'] = df['trade_direction'].map({1:'Long',-1:'Short'})
    df['trades'] = df_trades['trade']
    df_trades['trade']
    #Label our opening trade after removing our window period
    df['trades'].replace(pandas.np.NaN,'None',inplace=True)
    df = df.dropna()
    df['trades'].iloc[0]=('Long' if df['trade_direction'].iloc[0]==1.0 else 'Short')
    
    trading_strategy = AccountManager(
    symbol=symbol,
    trades_df=df,
    starting_capital=starting_capital,
    allocation_amount=allocation)

    results = trading_strategy.run()
    
    r_df = results['result_df']


    # Calculate the Absolute Return of our trend-following trading strategy
    absolute_return_tf = (r_df['account_val'].iloc[-1]-starting_capital)/starting_capital


    # Calculate the CAGR or our trend-following trading strategy
    CAGR_tf = ((r_df['account_val'].iloc[-1] /starting_capital)**(252/len(df)))-1



    # Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
    information_ratio_tf = (r_df.account_val.pct_change().mean()/r_df.account_val.pct_change().std())*((252) **0.5)

    absolute_return_bh = (r_df.Close.iloc[-1]-r_df.Open.iloc[0])/r_df.Open.iloc[0]

    # Calculate the CAGR or our trend-following trading strategy
    CAGR_bh = ((r_df.Close.iloc[-1]/r_df.Close.iloc[0])**(252/len(df)))-1

    # Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
    information_ratio_bh = (r_df.Close.pct_change().mean()/r_df.Close.pct_change().std())*((252) **0.5)

    #Plot the equity curve (hint: use 'close' returns and your initial capital value to generate an equity curve)


    print(f"Trend Follow: Absolute_return % {np.round(absolute_return_tf,2)*100}")
    print(f"Buy-and-Hold: Absolute_return % {np.round(absolute_return_bh,2)*100}")

    print(f"Trend Follow: CAGR % {np.round(CAGR_tf,2)*100}")
    print(f"Buy-and-Hold: CAGR % {np.round(CAGR_bh,2)*100}")

    print(f"Trend Follow: information_ratio {np.round(information_ratio_tf,4)}")
    print(f"Buy-and-Hold: information_ratio {np.round(information_ratio_bh,4)}")
    

    #Plot the equity curve (hint: use 'account_val')
    r_df.account_val.iplot(title=f'{symbol} Trend Following Equity Curve')
    ((r_df.Close.pct_change().dropna()+1).cumprod()*starting_capital).iplot(title=f'{symbol} Buy-and-Hold Equity Curve')

In [16]:
fun_with_assignment_2(symbol='CHZ-USD',
                      allocation=1,
                      starting_capital=1000,
                      short_trend=3,
                      long_trend=6,
                      start_date='2021-10-25',
                      end_date=None)

[*********************100%***********************]  1 of 1 completed


Trend Follow: Absolute_return % -81.0
Buy-and-Hold: Absolute_return % 33.0
Trend Follow: CAGR % -100.0
Buy-and-Hold: CAGR % 610.0
Trend Follow: information_ratio -5.9719
Buy-and-Hold: information_ratio 1.8813


In [17]:
fun_with_assignment_2(symbol='AVAX-USD',
                      allocation=1,
                      starting_capital=500,
                      short_trend=6,
                      long_trend=12,
                      start_date='2021',
                      end_date=None)

[*********************100%***********************]  1 of 1 completed


Trend Follow: Absolute_return % 636.0
Buy-and-Hold: Absolute_return % 2078.0
Trend Follow: CAGR % 391.0
Buy-and-Hold: CAGR % 1079.0
Trend Follow: information_ratio 1.7578
Buy-and-Hold: information_ratio 2.3019
